In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [4]:
spark = SparkSession.builder.appName('linreg').getOrCreate()

In [5]:
# Get data
data = spark.read.csv('Ecommerce_Customers.csv',
                      header=True,inferSchema=True)

In [7]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [14]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [15]:
# Setup the dataframe for Machine Learning
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [16]:
# We only want numerical data
# Predict yearly amount spent, based on all other features
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [17]:
# Create a vector assembler with the features you want
assembler = VectorAssembler(inputCols=['Avg Session Length',
                                       'Time on App',
                                       'Time on Website',
                                       'Length of Membership',],
                            outputCol='features')

In [20]:
# Pass your data through the assembler to transform it
output = assembler.transform(data)

In [22]:
output.printSchema() # new features column!

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [23]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [24]:
# Final dataframe for Machine Learning
final_data = output.select('features','Yearly Amount Spent')

In [25]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [26]:
# Now, split the data
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [27]:
# Double-Check the split
train_data.describe().show()
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                350|
|   mean|   499.693822310208|
| stddev|  79.12777788097375|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                150|
|   mean| 498.42787547148396|
| stddev|  80.00828607335022|
|    min| 298.76200786180766|
|    max|  744.2218671047146|
+-------+-------------------+



In [29]:
# Create a LinReg model, fit to train_data and test on test_data
lr = LinearRegression(featuresCol='features',
                      labelCol='Yearly Amount Spent')

In [30]:
lrModel = lr.fit(train_data)

In [31]:
test_results = lrModel.evaluate(test_data)

In [32]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -12.11640169183454|
| -5.240509344014697|
|  -4.60904175202694|
|  4.825688205101528|
|-1.4059948798309279|
|-4.7029750299532225|
| -7.399714290909628|
| -8.904405016292174|
| 3.5882205093180914|
| 1.7893011177555422|
| -2.813679750077597|
|  7.509780164472147|
| 0.9107984148354262|
|  7.533224973853123|
|-10.994219448615354|
| -9.025152365002612|
| -2.276156743950139|
|-14.031856988132631|
|  5.003853277066241|
|  8.231003179699087|
+-------------------+
only showing top 20 rows



In [33]:
test_results.rootMeanSquaredError

9.937266574363559

In [36]:
test_results.r2 # 98.4% goodness-of-fit

0.9844700900709007

In [37]:
final_data.describe().show()
# We have a variation of roughly 10 dollars on the Yearly
# Amount Spent. Avg is 500, so it is pretty good

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [38]:
# Can we trust such a good model? Let's check!
# Deploy the model on unlabeled data

In [39]:
unlabeled_data = test_data.select('features')

In [40]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.8364326747734...|
|[30.8794843441274...|
|[31.0472221394875...|
|[31.2606468698795...|
|[31.4252268808548...|
|[31.4474464941278...|
|[31.5261978982398...|
|[31.5316044825729...|
|[31.7366356860502...|
|[31.8186165667690...|
|[31.8209982016720...|
|[31.8293464559211...|
|[31.8512531286083...|
|[31.8648325480987...|
|[31.8854062999117...|
|[31.9120759292006...|
|[31.9365486184489...|
|[31.9480174211613...|
|[31.9549038566348...|
+--------------------+
only showing top 20 rows



In [41]:
predictions = lrModel.transform(unlabeled_data)

In [43]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|332.04527149502815|
|[30.8364326747734...| 472.7424097710043|
|[30.8794843441274...| 494.8156417368816|
|[31.0472221394875...|387.67171098391987|
|[31.2606468698795...| 422.7326261367823|
|[31.4252268808548...| 535.4696936847151|
|[31.4474464941278...|426.00245638613364|
|[31.5261978982398...|   417.99893120863|
|[31.5316044825729...|432.92738522004447|
|[31.7366356860502...| 495.1441451377764|
|[31.8186165667690...|449.23235312021325|
|[31.8209982016720...| 417.1655008487412|
|[31.8293464559211...|384.24153957313956|
|[31.8512531286083...|465.45902169294527|
|[31.8648325480987...|450.88549992542903|
|[31.8854062999117...| 399.1284253374781|
|[31.9120759292006...|389.81087304965786|
|[31.9365486184489...| 441.2312418834608|
|[31.9480174211613...|456.91702361583157|
|[31.9549038566348...| 431.7668767602279|
+--------------------+------------